# Introduction to Chameleon

- **Estimated time**: 45 minutes
- **Requirements**:
  - A user account on Chameleon, with an active project allocation

This tutorial will teach you the basics of using a cloud computing environment as a platform for Computer Science research. Besides introducing you to the Chameleon testbed, you will learn some basic cloud computing concepts, which you can apply to any cloud provider, such as [AWS](https://aws.amazon.com/), [Google Cloud](https://cloud.google.com/), [Azure](https://azure.microsoft.com/en-us/), and others.

## Background: Cloud Computing for Experimentation

Using the cloud for running experimentation, simulation, and other scientific tasks requires thinking about things a bit differently. While ultimately the hardware that is running your code may physically reside in a data center similar to one you may already be using in a national laboratory or academic instutition, the _interface_ to the compute resources tends to be quite different. For one, cloud environments allow for **high configurability**, often allowing you to create arbitrary network topologies, provision machines with varied locality (e.g. the same physical rack, or entirely different continents), and access a wide variety of heterogenous hardware, all via a unified remote management interface.

Clouds are typically multi-tenant; multiple users are utilizing a pool of resources and sharing them, even if they often might not realize or think about this. In order to partition a cloud's resources, whether it's networking, compute cores, or storage/memory, several abstraction patterns have emerged that tend to be the same across any cloud provider. You could consider them to be a cloud's _interface_: a cloud necessarily should provide these resources to be a cloud. Some of the most important (and common) patterns are:

- **Bootable disk images**: disk images with an OS, usually preconfigured with some "main" software stack or otherwise tuned for a specific purpose, like a MySQL database, or a Hadoop node.
  - AWS equivalent: [AMI](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AMIs.html)
  - Chameleon equivalent: [Image](https://chameleoncloud.readthedocs.io/en/latest/technical/images.html)
- **Orchestration templates**: a combination of disk images, with rules for how to deploy those disk images in a certain configuration, usually involving other aspects of the cloud, such as networking or storage.
  - AWS equivalent: [CloudFormation](https://aws.amazon.com/cloudformation/).
  - Chameleon equivalent: [Complex Appliance (Heat)](https://chameleoncloud.readthedocs.io/en/latest/technical/complex.html)
- **Virtualized networking**: a conceptual model of a network is usually expressed with three primary domain entities:
  1. _Network_: the top-level resource. This can be a layer-2 concept and include a VLAN tag, or just be a "holder" for the other domain entities.
    - AWS equivalent: [VPC](https://aws.amazon.com/vpc/)
    - Chameleon equivalent: [Network](https://chameleoncloud.readthedocs.io/en/latest/technical/networks/networks_vlan.html#creating-a-network-using-the-gui)
  1. _Subnet_: a segment of the network used to partition a larger network into smaller units.
    - AWS equivalent: [VPC Subnet](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Subnets.html)
    - Chameleon equivalent: [Subnet](https://chameleoncloud.readthedocs.io/en/latest/technical/networks/networks_vlan.html#creating-a-network-using-the-gui)
  1. _Router_: a set of rules used to forward traffic between various subnets.
    - AWS equivalent: [VPC Route Table](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Route_Tables.html)
    - Chameleon equilvanet: [Router](https://chameleoncloud.readthedocs.io/en/latest/technical/networks/networks_vlan.html#creating-a-router)
- **Object storage**: a shared and highly-available storage backend used for storing blobs of data. This is different than a block storage device in that it represents files as opaque entities; they are not a series of blocks that you can access. Usually object storage is used for storing various cloud entities, such as disk images or orchestration templates, but any arbitrary data can be stored here as well, such as experiment results or input data sets.
  - AWS equilvanet: [S3](https://aws.amazon.com/s3/)
  - Chameleon equilvanet: [Object Store (Swift)](https://chameleoncloud.readthedocs.io/en/latest/technical/swift.html)
- **Key pairs**: public-key cryptography is used to allow access to compute resources; users therefore are given a way to manage which public keys can be used to authenticate them when they log on. (This is a separate authentication mechanism to the user's cloud username/password, which is used to log in to the cloud provider itself.)
  - AWS equivalent: [EC2 Key Pair](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html)
  - Chameleon equivalent: [Key Pair](https://chameleoncloud.readthedocs.io/en/latest/technical/gui.html#key-pairs)

### What is Chameleon?

Chameleon is a highly configurable environment for large-scale cloud research. Not only is Chameleon suitable for research in to how clouds can be improved and built, Chameleon is also implemented as a cloud itself! Therefore, it adheres to the _interface_ defined in the previous section, as you can see.

Chameleon is unique in that it offers a limited amount of highly-specialized heterogenous hardware, and also a (less-limited) cluster of commodity machines. Due to the specialized nature of the machines available in the cloud, sometimes the demand exceeds the supply. In order to provide fairness to the users of Chameleon, users must therefore make an _advanced reservation_ for the resources required. This is a notable difference from commercial cloud providers such as AWS, which have a much larger inventory of relatively homoegenous hardware. Please keep this in mind as you use Chameleon, and when you no longer require the resources you've reserved, release them back in to the pool by ending your reservation.

Another unique aspect of Chameleon is that it provisions exclusively on _bare metal_. This means there is no multi-tenancy on the hardware you reserve; it is for your exclusive use. All of the Chameleon hardware is provisioned in this fashion. Though some cloud providers support limited bare metal use cases (e.g. AWS [bare metal instance types](https://aws.amazon.com/about-aws/whats-new/2018/05/announcing-general-availability-of-amazon-ec2-bare-metal-instances/)), most only support virtualized environments, e.g. VMs.

### The experimental workflow

This tutorial is structured around a framework we refer to as the **Experimental Workflow**. A typical experiment consists of several phases. First, you **design the experiment**; you have some idea of what you would like to test. Next, you figure out what **resources** exist to help you instantiate your experiment. In a cloud environment, this step is important to call out, because likely the cloud has many different types of resources available to you, and it is your task to figure out which are the most relevant to your needs. Once you have found suitable resources, you **provision** them--this means, you set up or otherwise "activate" them for your experiment. Then, the main part of your experiment setup begins, where you **configure** the resources as needed to run your experiment. This is typically an **interactive** step, at least initially: you are discovering how to run your experiment optimally. This is also the step where you get your hands dirty and start running tests! Once your experiment is running, you **monitor** the state of the experiment and results generated. Finally, armed with your experimental data, you **analyze**, **discuss** the results, and hopefully **share** them with the wider community. This often leads to you refining your experiment, or having new ideas for new experiments, and the cycle begins again.

<br>
<center>
    <img src="./images/ExperimentalWorkflow.png"><br>
    <em>The Experimental Workflow</em>
</center>
<br>

For the purposes of this tutorial, we'll go over the bits relevant to the cloud (in our case, Chameleon), which are colored green in the above chart. We'll go over **discovering resources** via the Chameleon Hardware Catalog, **provisioning resources** by reserving bare metal nodes and launching server instances on them, **configuring and interacting** with our instances via SSH, and finally **monitoring** the state of our experiment via metrics exported automatically from our server instances. Let's get started!

## Tutorial

In this tutorial you'll primarily be interacting with Chameleon's GUI. We'll cover how each step of the _experimental workflow_ is supported by Chameleon.

1. [Discover resources](#Step-1%3A-Discover-resources)
1. [Provision resources](#Step-2%3A-Provision-resources)
1. [Configure and interact](#Step-3%3A-Configure-and-interact)
1. [Monitor](#Step-4%3A-Monitor)

## Step 1: Discover resources


### The Chameleon hardware catalog

Chameleon has a [browseable catalog of hardware available for reservation](https://www.chameleoncloud.org/hardware/). You can filter the list of resources available across many dimensions. The big buttons near the top allow easily filtering by a specific "node type". Most nodes of the same type have the same properties, but there are sometimes nodes within a type that have extra capabilities, such as an additional network card active. There are additional common filters displayed immediately below that allow you to e.g. filter by nodes only at a specific regional site. You can expand the advanced filters below to see even more options. If a filter does not apply (because all filtered nodes have the same value for this option), it will be disabled.

Once you have narrowed your search down a bit, you can click the "View" button below to see the list of nodes, with all of their details, that matched your query.

This can be helpful if you want to know specifics about the types of hardware available for experimentation. In particular, you can see what types of CPU processors are on the node, as well as how many disk drives it has, how much memory, and any other particularities about the node that could make it an interesting candidate for your experiment.

<center>
  <video src="https://chi.uc.chameleoncloud.org:7480/swift/v1/chameleon-tutorials-public/IntroductionToChameleon/hardware_discovery_480p.webm" controls></video><br>
  <em>An overview of the Hardware Discovery portal</em>
</center>

### Types of bare metal nodes available on Chameleon

A few practical notes about the [node types](https://chameleoncloud.readthedocs.io/en/latest/technical/reservations.html#chameleon-node-types):

- `compute_skylake`: Besides having an Intel Skylake processor, these nodes are great options for those who are looking to perform SDN experiments, because their primary network interface is connected to a Corsa DP2000 switch, which gives you a virtual router that can have its control plane be controlled by you.
- `compute_haswell`: These are the workhorse nodes on Chameleon, and have Intel Haswell processors. They are the most plentiful, so are good candidates for large cluster experiments. They also have two NICs available instead of one, so there are some possibilities for more advanced networking topologies here.
- `storage`, `storage_hierarchy`: As expected, these nodes have a _lot_ of storage space. The `storage_hierarchy` nodes have the storage split between memory, NVMes, SSDs, and HDDs, allowing for interesting experiments around caching strategies.
- `compute_haswell_ib`: These are special Haswell nodes that are also connected via InfiniBand.
- `gpu_*`: These nodes all have at least one NVIDIA GPU on board.

### Using the hardware API

If you like, you can interact with the hardware catalog over an HTTP/JSON API as well! For more information, please refere to the [Resource Discovery API](https://chameleoncloud.readthedocs.io/en/latest/technical/discovery.html#using-the-rest-apis-for-resource-discovery) documentation. Here is an example of what is possible:

In [ ]:
CHAMELEON_SITE=tacc

all_nodes=$(curl -sS https://api.chameleoncloud.org/sites/$CHAMELEON_SITE/clusters/chameleon/nodes)

echo "Found $(jq '.items | length' <<<$all_nodes) nodes in total!"

echo "Here are the first 10 nodes of type 'compute_skylake':"
jq -r '.items
  | map(select(.node_type == "compute_skylake"))
  | map(.uid)[0:10][]' <<<$all_nodes

### The Chameleon appliance catalog

Chameleon also provides a set of preconfigured disk images and orchestration templates (what Chameleon calls [Complex Appliances](https://chameleoncloud.readthedocs.io/en/latest/technical/complex.html).) In particular, Chameleon maintains stable disk images with the most current release of [CentOS7](https://www.chameleoncloud.org/appliances/1/) and [Ubuntu16.04](https://www.chameleoncloud.org/appliances/19/) (Ubuntu18.04 support forthcoming). There are also images and orchestration templates contributed by other Chameleon users for your use! Please take a moment to browse the catalog, and if you find that you're creating something yourself that could benefit the wider Chameleon community, please [publish it!](https://chameleoncloud.readthedocs.io/en/latest/technical/complex.html#sharing-complex-appliances). Browse our entire Appliance Catalog [here](https://www.chameleoncloud.org/appliances/).

<center>
    <video src="https://chi.uc.chameleoncloud.org:7480/swift/v1/chameleon-tutorials-public/IntroductionToChameleon/appliance_catalog_480p.webm" controls></video><br>
    <em>The Chameleon Appliance Catalog</em>
</center>



## Step 2: Provision resources

Now that you're familiar with what's available on Chameleon, let's talk about how you actually get access to these resources, via _reservations_. We're going to make a reservation on Chameleon for a bare metal node. You should see embedded videos in each step, which should help you navigate the user interface.

### Log in to the bare metal experimentation site

1. Log in at [chameleoncloud.org](https://www.chameleoncloud.org).
1. At the top-right, click on your username, then click "Experiment" in the drop-down menu.
1. Click the "Bare metal site" link on this page. You will be taken to a new page (you may be asked to wait for a short time), and eventually will land on the bare metal site.

<center>
    <video src="./videos/login_480p.webm" controls></video><br>
    <em>Logging in to the Bare metal site</em>
</center>

### Select a project and region

1. Click the dropdown immediately to the right of the Chameleon logo in the header. This displays what project is currently selected, and which Chameleon region you have selected (e.g. TACC or UC).
1. In the dropdown, change your project or region by clicking one of the options listed. If you do not see your project here, ask your PI to [add you as a member to the project](https://chameleoncloud.readthedocs.io/en/latest/technical/project.html#manage-users).

<center>
    <video src="https://chi.uc.chameleoncloud.org:7480/swift/v1/chameleon-tutorials-public/IntroductionToChameleon/select_project_480p.webm" controls></video><br>
    <em>Selecting an active project</em>
</center>

### Make a reservation

1. On the left-hand sidebar, click "Reservations" and then "Leases" to open the [Leases panel](https://chi.tacc.chameleoncloud.org/project/leases/).
1. This panel displays all leases currently pending or active (or already terminated) for your project. Click "Create Lease" to open the lease creation modal.
1. The only field required in this modal is the lease name, but you likely want to fill in other details. You can specify the time range (note that you are limited to 7 days maximum; for more information on lease restrictions visit our [FAQ](https://www.chameleoncloud.org/about/frequently-asked-questions/#toc-are-there-any-limitations-of-chameleon-usage-)), the start date (to queue up a lease in the future), whether to reserve bare metal machines, network resources, or both, and fine-tune the bare metal node selection with filters.
1. When you are satisfied, click "Create Lease". If the lease is successfully created, the lease list will refresh and you should see your lease in the "PENDING" state. It should transition to the "STARTING" state shortly after its start time has passed.

<center>
    <!-- Set size to 640 (480p) for consistency -->
    <video src="https://chi.uc.chameleoncloud.org:7480/swift/v1/chameleon-tutorials-public/IntroductionToChameleon/reservations_540p.webm" controls width="640"></video><br>
    <em>Making a reservation for a bare metal node</em>
</center>

### Launch a bare metal instance

1. Click "Compute", then "Instances" in the left-hand sidebar to open the [Instances panel](https://chi.tacc.chameleoncloud.org/project/instances/).
1. This panel will list all instances currently running, as well as any in the process of launching. To launch a new instance, click the "Launch Instance" button to open the Launch Instance modal.
1. This modal has many options, but you likely are only concerned with a few basic options. The instance name is required. You must also specify which reservation to launch the instance under; this determines which set of nodes will be used when picking one to host the instance. The lease name will display in this drop-down to help you pick.
1. Click the "Source" tab on the left to choose the image you wish to launch. If this is your first time, we recommend using the CC-CentOS7 image. You can filter the list of images with the filter box. **Note**: it may take a while to display the list of images. You may need to wait a few seconds for the entire list to show up.
1. Click the "Flavor" tab and click the up-arrow icon to select the "baremetal" flavor, which is the only option.
1. Click the "Networks" tab and similarly, select a network to attach to the node. **Note**: you can only attach as many networks as you have NICs enabled on the node. For most nodes, there is only 1 NIC enabled, so only 1 network can be selected.
1. Click the "Key Pair" tab and select which SSH keys with which you wish to allow authenticating to the instance over SSH. **This is important to do, otherwise you will not be able to access your instance!**
1. Click "Launch Instance" to trigger the build of your instance with these parameters. You should see your image in the "Build" state, and it should transition through a few stages, such as "Scheduling", "Networking", and "Spawning", before arriving in the "Active" state. This entire process takes about 10 minutes on average, but can be higher if the system is experiencing high load.

<center>
    <!-- Set size to 640 (480p) for consistency -->
    <video src="https://chi.uc.chameleoncloud.org:7480/swift/v1/chameleon-tutorials-public/IntroductionToChameleon/create_instance_540p.webm" width="640" controls></video><br>
    <em>Launching an instance on a reserved bare metal node</em>
</center>

## Step 3: Configure and interact

### Assign a floating (public) IP

1. Click "Network", then "Floating IPs" in the left-hand sidebar to open the [Floating IPs panel](https://chi.tacc.chameleoncloud.org/project/floating_ips/).
1. This panel will list any Floating IPs currently assigned to your project for use. Floating IPs that you reserved as part of a lease will appear here. If there are other Floating IPs free on the testbed, you can allocate them to your project by clicking the "Allocate Floating IP" button, then clicking "Allocate" in the modal that displays.
1. To assign a Floating IP to a running instance, click the "Associate" button to the right of the Floating IP address you would like to use. The "Manage Floating IP Associations" modal dialog will display.
1. In the Port dropdown, select the instance you want to assign the Floating IP to. If your instance has multiple networks assigned, it will appear multiple times in this list and you should choose which interface (port) to assign the Floating IP to. Click "Associate" to complete the process.
1. Once the Floating IP is assigned, it will display in the Instances panel as one of your instance's IP addresses. You should be able to connect to this IP address over the public Internet. It may take a minute or two for the routing to propagate through the system.

<center>
    <!-- Set size to 640 (480p) for consistency -->
    <video src="https://chi.uc.chameleoncloud.org:7480/swift/v1/chameleon-tutorials-public/IntroductionToChameleon/assign_floating_ip_540p.webm" width="640" controls></video><br>
    <em>Assigning a Floating IP address</em>
</center>

## Step 4: Monitor

First, we need to figure out the UUID of your instance. This can be done in a number of ways; you can find it in the Instance details page when you click on your Instance name, or you can find it by using the `openstack server show` CLI command. We'll attempt to automatically find it in the next cell.

In [ ]:
SERVER_NAME="my-first-instance"

for region in "CHI@TACC" "CHI@UC"; do
  SERVER_UUID="$(openstack --os-region "$region" server show "$SERVER_NAME" -f value -c id)"
  if [[ "${SERVER_UUID:+x}" == "x" ]]; then
    if [[ "$region" != "$OS_REGION_NAME" ]]; then
      echo "This server was found in a region different than the region"
      echo "set in your Notebooks' environment (as the \$OS_REGION_NAME variable)."
      echo "Setting the OS_REGION_NAME variable to $region to match..."
      export OS_REGION_NAME="$region"
    fi
    break
  fi
done

if [[ "${SERVER_UUID:+x}" != "x" ]]; then
  echo "Could not find server $SERVER"
else
  echo "Found server with ID $SERVER_UUID"
fi

### Collecting metrics for your instance

Each pre-configured Chameleon disk image includes an agent that automatically registers metrics that track various aspects of your node throughout its deployment, such as memory usage, load, and network interface usage. You can query these metrics via the `openstack metric` subcommands. For more information, please refer to the [Monitoring](https://chameleoncloud.readthedocs.io/en/latest/technical/metrics.html) section of the documentation. 

In [ ]:
echo "Available metrics for $SERVER_UUID:"
openstack metric resource show "$SERVER_UUID" -f json \
  | jq -r '.metrics | to_entries[].key' | sort | sed 's/^/  - /'

METRIC_NAME="load@load"
METRIC_GRANULARITY=60 # Possible values: 1, 60, 3600 (seconds, minutes, hours)

echo
echo "Getting values for $METRIC_NAME (${METRIC_GRANULARITY}-second granularity) ..."
openstack metric measures show \
  --resource-id "$SERVER_UUID" \
  --granularity $METRIC_GRANULARITY \
  --refresh \
  "$METRIC_NAME" \
  | head -n 10
echo "... (rest clipped) ..."

#### CSV output

The output can also be saved as a CSV file, for easy analysis in another Notebook, or elsewhere! Use the `--format` option to select what format the data should be outputted in. Most `openstack` CLI commands support choosing various output formats, such as CSV, table, and JSON.

In [ ]:
filename="./out/$SERVER_UUID-$METRIC_NAME-$METRIC_GRANULARITY-$(date +%Y%m%d).csv"
mkdir -p $(dirname $filename)

openstack metric measures show \
  --resource-id "$SERVER_UUID" \
  --granularity $METRIC_GRANULARITY \
  --refresh \
  "$METRIC_NAME" \
  --format csv >$filename

echo "Saved as $filename"

## Recap

We walked through how to find resources on Chameleon that match your experimental needs. Then, we made a reservation for a bare metal node on the testbed. We provisioned a stock CentOS 7 instance on that node, and assigned a Floating IP, which allowed us to log in. We explored how performance metrics can be downloaded from the instance.

### Outputs

Here are some output variables you may want to use in other tutorials:

In [ ]:
cat <<EOF
SERVER_UUID="$SERVER_UUID"
EOF